In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
template = """
&FORCE_EVAL                                  ! This section defines method for calculating energy and forces
  METHOD FIST                                ! Using Molecular Mechanics
  &MM
    &FORCEFIELD                              ! This section specifies forcefield parameters 
      parm_file_name ace_ala_nme.pot
! This file contains force field parameters
      parmtype CHM                        ! forcefield parameters has CHARMM format
      &SPLINE                ! This section specifies parameters to set up the splines used
! in the nonboned interactions (both pair body potential and many body potential)
        EMAX_SPLINE 1.0                        ! Specify the maximum value of the potential up to which splines will be constructed
      &END
    &END FORCEFIELD
    &POISSON                                ! This section specifies parameters for the Poisson solver
      &EWALD                                ! This section specifies parameters for the EWALD summation method (for the electrostatics)
        EWALD_TYPE ewald                    ! Standard non-fft based ewald method
        ALPHA .36
        GMAX 29                             ! Number of grid points
      &END EWALD
    &END POISSON
    &PRINT                                  ! This section controls printing options
      &FF_INFO
      &END
      &FF_PARAMETER_FILE
      &END
    &END
  &END MM
  &SUBSYS                                   ! This section defines the system 
    &CELL                                   !  Unit cell set up
      ABC 50.0 50.0 50.0                    ! Lengths of the cell vectors A, B, and C
    &END CELL
    &COLVAR                                 ! This section specifies collective varialbe
       &TORSION                             ! This section defines variable as torsion angle
         ATOMS 5 7 9 15                     ! Four atoms specify torsion angle
       &END
       &PRINT
       &END
    &END
    &COLVAR                                 ! This section specifies collective varialbe
       &TORSION                             ! This section defines variable as torsion angle
         ATOMS 7 9 15 17                    ! Four atoms specify torsion angle
       &END
       &PRINT
       &END
    &END
     &TOPOLOGY
      CONN_FILE_NAME ace_ala_nme.psf
! File which contains the connectivity information
      CONNECTIVITY psf                      ! Format of the connectivity file is PSF
      COORD_FILE_NAME ini.pdb
! File which contains atom's coordinates of the system
      COORDINATE PDB                        ! Coordinates are in the PDB format
      &DUMP_PDB                             ! This sections specifies the dumping of the PDB at the starting geometry
      &END
      &DUMP_PSF                             ! This sections specifies the dumping of the PSF connectivity
      &END
    &END TOPOLOGY
    &PRINT
      &TOPOLOGY_INFO
        AMBER_INFO
      &END
    &END
  &END SUBSYS
&END FORCE_EVAL

&GLOBAL                                  ! Section with general information regarding which kind of simu lation to perform an parameters for the whole PROGRAM
  PRINT_LEVEL LOW                        ! Global print level
  PROJECT ch                             ! Name of the project. This word will appear as part of a name of all ouput files (except main ouput file, specified with -o option)
  RUN_TYPE GEO_OPT                       ! Run type is a geometry optimization
&END GLOBAL
&MOTION                                  ! This section specifies a set of tool connected with the motion of the nuclei
 &CONSTRAINT                             ! Section specifying information regarding how to impose constraints on the system
  &COLLECTIVE
  INTERMOLECULAR T
    COLVAR 1                             ! Sequential number of the variable
    &RESTRAINT                           ! This section specifies how stong is the restraint
      K=5.0                              ! U(x)=K*(x-x0)^2
    &END
    TARGET [deg] %f                      ! _A1_ will be changed to the number by an external scritp
  &END
   &COLLECTIVE
  INTERMOLECULAR T
    COLVAR 2                             ! Sequential number of the variable
    &RESTRAINT                           ! This section specifies how stong is the restraint
      K=5.0                              ! U(x)=K*(x-x0)^2
    &END
    TARGET [deg] %f                      ! _A1_ will be changed to the number by an external scritp
  &END
 &END

 &PRINT                                  ! This section controls the printing properties during an optimization or MD run
   &TRAJECTORY
    FORMAT PDB                           ! Format of  the ouput trajectory is PDB
    ADD_LAST NUMERIC                     ! (Wiki) If the last iteration should be added, and if it should be marked symbolically (with l) or with the iteration number.Not every iteration level is able to identify the last iteration early enough to be able to output. When this keyword is activated all iteration levels are checked for the last iteration step. 
    &EACH
     GEO_OPT 100                         ! Print one frame every 100 iteration
    &END
   &END
 &END
 &GEO_OPT                                ! This section specifies optimizer options
    OPTIMIZER BFGS                       ! Type of the optimizer 
    MAX_ITER 5000                        ! Maximum number of optimization steps
    MAX_FORCE 0.005                      ! The value of maximal force
    RMS_FORCE 0.003                      ! The value of maximal force RMS
 &END
&END

"""

In [ ]:
a1a2ene = []

! cp ini0.pdb ini.pdb
for i in range(5):
    a1 = -180 + 80 * i
    for j in range(5):
        print("%i%i "%(i,j), end='')
        
        a2 = -180 + 80 * j
        inp = template% (a1, a2)
        open("energy.inp", "w").write(inp)
        ! cp2k.popt energy.inp > energy.out
        
        # saving the optimized geometry (last 25 lines of the optimization trajectory file)
        #into opt.1.1.pdb (if i=1 and j=1)
        #TODO tail -25 ch-pos-1.pdb > opt.$i.$j.pdb
        
        colvars_lines = ! grep -A 2 '&COLVAR_RESTART' ch-1.restart
        af1 = float(colvars_lines[1])
        af2 = float(colvars_lines[2])
        energy_lines = ! grep "ENERGY|" energy.out
        energy = float(energy_line[-1].split()[-1])
        a1a2ene.append((af1, af2, energy))              

In [ ]:
# TODO plot
a1a2ene = np.array(a1a2ene)
plt.contour(a1a2ene[:,0], a1a2ene[:,1], a1a2ene[:,2])

In [ ]:
# #!/bin/bash					
# rm a1a2ene							# rm the file a1a2ene
# cp ini0.pdb ini.pdb						# cp file ini0.pdb to ini.pdb
# for i in {0..4}							# external loop for i changes from 0 to 4
# do
# 	a1=`echo -180 + 80 \* $i | bc -l `				# performs a1=-180+80*i. Where bc is a calculator
# 	for j in {0..4}							# internal loop for j changes from 0 to 4
# 	do
# 		a2=`echo -180 + 80 \* $j | bc -l `			# performs a2=-180+80*j. Where bc is a calculator
# 		echo $a1 $a2
# 		sed "s/_A1_/$a1/g;s/_A2_/$a2/g" < inp.templ > inp.$i.$j	# Reading file inp.templ and performing changes: _A1_ -> (value of) a1 and _A2_ -> (value of) a2. All changes will be saved in the file inp.1.1 (if i=1 and j=1)
# 		#    ^ s:substitute
# 		#	^ _A1_	will be substituted by (the value of) a1
# 		#	     ^  will substitute _A1_
# 		#		^ g: perform changes globally
# 		#		   ^s:substitute
# 		#		      ^ _A2_ will be substituted by (the value of) a2
# 		#			  ^ will substitute _A2_
# 		#			     ^g: perform changes globally
# 		#mpiexec -np ${NSLOTS} /home/apps/cp2k/bin/cp2k.popt -i inp.$i.$j > out.$i.$j
# 		cp2k.popt -i inp.$i.$j -o out.$i.$j			# running cp2k
# 		tail -25 ch-pos-1.pdb > opt.$i.$j.pdb			# saving the optimized geometry (last 25 lines of the optimization trajectory file) into opt.1.1.pdb (if i=1 and j=1)
# 		cp opt.$i.$j.pdb ini.pdb				# copying the optimized geometry into the file ini.pdb in order to use it as starting point for the next optimization
# 		af1=`grep -A 2 '&COLVAR_RESTART' ch-1.restart | head -2 | tail -1 `	# getting the value of the phi angle
# 		af2=`grep -A 2 '&COLVAR_RESTART' ch-1.restart | tail -1 ` 		# getting the value of the psi angle
# 		ef=`grep 'ENERGY|' out.$i.$j  | tail -1 | awk '{print $9}' `
# 		echo $af1 $af2 $ef >> a1a2ene				# adding the line "af1 af2 $ef" at the end of the file a1a2ene	
# 	done								
# 	echo " " >> a1a2ene						# adding a blanc line at the end of the file when internal loop finishes
# done

